In [2]:
import torch
import pandas as pd
import numpy as np
from pylab import mpl, plt
plt.style.use('seaborn')

from torch.utils.data.sampler import SequentialSampler
from torch.utils.data.dataloader import DataLoader
from sklearn.preprocessing import MinMaxScaler
# from torch.utils.data import Dataset


print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
1
NVIDIA GeForce RTX 2060


## Load date

- `class ClosePriceDataset()`: `Dataset`
    - `__init__`: read_csv()
        - `MinMaxScaler`: [-1, 1]，(可以被reverse回原值)
- `def train_valid_split`: return `Sampler`, split training data(80%) and validation data(20%)
- `train_loader`: return `DataLoader`
- `val_loader`: return `DataLoader`

In [19]:
from operator import index


dates = pd.date_range('2010-01-02','2017-10-11', freq='B')
df_dates = pd.DataFrame(index=dates)
df_ibm = pd.read_csv("../datasets/ibm.csv", parse_dates=True, index_col='Date')
df1 = df_ibm.join(df_dates, how='inner')
df2 = df_dates.join(df_ibm)
df1.info()
df2.info()
# df_ibm.head()
# df_ibm[['Close']].plot(figsize=(10, 6))
# plt.title = 'IBM Stock Price'
# plt.xlabel = 'Date'
# plt.ylabel = 'Price'
# plt.show()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1958 entries, 2010-01-04 to 2017-10-11
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Open     1958 non-null   float64
 1   High     1958 non-null   float64
 2   Low      1958 non-null   float64
 3   Close    1958 non-null   float64
 4   Volume   1958 non-null   int64  
 5   OpenInt  1958 non-null   int64  
dtypes: float64(4), int64(2)
memory usage: 107.1 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2028 entries, 2010-01-04 to 2017-10-11
Freq: B
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Open     1958 non-null   float64
 1   High     1958 non-null   float64
 2   Low      1958 non-null   float64
 3   Close    1958 non-null   float64
 4   Volume   1958 non-null   float64
 5   OpenInt  1958 non-null   float64
dtypes: float64(6)
memory usage: 175.4 KB


## Model

每一層需給定的input（計算上一層output）的size

input: (batch-size, channel, data-length)
(32, 1, 60)


In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class ConvNet(nn.Module):
    """Some Information about MyModule"""
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, groups=1, bias=True, kernel_size=20, stride=1)
        self.maxpool1 = nn.MaxPool1d(kernel_size=5)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=16, groups=1, bias=True, kernel_size=5, stride=1)
        self.maxpool2 = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(2 * 16, 50)
        self.fc2 = nn.Linear(50, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        # print('conv1 Shape: {}'.format(x.shape))
        x = self.maxpool1(x)
        # print('MaxPool1 Shape: {}'.format(x.shape))
        x = F.relu(self.conv2(x))
        # print('conv2 Shape: {}'.format(x.shape))
        x = self.maxpool2(x)
        # print('MaxPool2 Shape: {}'.format(x.shape))
        x = torch.flatten(x, start_dim=1)
        # print('Flaten Shape: {}'.format(x.shape))
        x = F.relu(self.fc1(x))
        # print('Linear1 Shape: {}'.format(x.shape))
        x = self.fc2(x)
        
        return x

net = ConvNet()
print(net)

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters())

ConvNet(
  (conv1): Conv1d(1, 64, kernel_size=(2,), stride=(1,))
  (maxpool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=128, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=1, bias=True)
)


## Training

丟進model需要為`Tensor`型態. 

inputs, labels: `Tensor`

In [4]:
epochs = 100
hist = np.zeros(epochs)
for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        # inputs = torch.from_numpy(x)
        # labels = torch.from_numpy(y)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward() # calculate dl/dw
        optimizer.step() # update weights

        # print statistics
        running_loss += loss.item()
        # if (i % (len(train_loader) // 4) == 0) or (i == len(train_loader)-1):    # print loss every 5 mini-batches
        #   print(f'[ epoch: {epoch + 1}, batch: {i + 1:5d}] loss: {running_loss / 5:.3f}')
        #   running_loss = 0.0
    
    print(f'[ epoch: {epoch + 1} ].   MSE: {running_loss}')
    hist[epoch] = running_loss
print('Finished Training')


plt.plot(hist, label="Training loss")
plt.legend()
plt.show()

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[1,     1] loss: 23056.034


/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([68])) that is different to the input size (torch.Size([68, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[2,     1] loss: 749.317
[3,     1] loss: 747.980
[4,     1] loss: 747.552
[5,     1] loss: 748.636
[6,     1] loss: 749.139
[7,     1] loss: 749.493
[8,     1] loss: 749.746
[9,     1] loss: 749.931
[10,     1] loss: 750.066
[11,     1] loss: 750.169
[12,     1] loss: 750.248
[13,     1] loss: 750.309
[14,     1] loss: 750.358
[15,     1] loss: 750.396
[16,     1] loss: 750.428
[17,     1] loss: 750.453
[18,     1] loss: 750.473
[19,     1] loss: 750.493
[20,     1] loss: 750.505
[21,     1] loss: 750.516
[22,     1] loss: 750.525
[23,     1] loss: 750.534
[24,     1] loss: 750.541
[25,     1] loss: 750.547
[26,     1] loss: 750.553
[27,     1] loss: 750.559


KeyboardInterrupt: 

## validation

計算validation data的RMSE

In [ ]:
import math

pred_value = np.zeros(len(val_loader))
true_value = np.zeros(len(val_loader))
loss = 0
for i, (inputs, labels) in enumerate(val_loader):
    pred = net(inputs)
    pred = scaler.inverse_transform(pred.detach().numpy())
    ans = scaler.inverse_transform(labels.reshape(-1, 1))
    pred_value[i] = pred
    true_value[i] = ans
    loss = (pred - ans) ** 2

print(loss/len(val_loader))


# Visualising the results
figure, axes = plt.subplots(figsize=(15, 6))
axes.xaxis_date()

axes.plot(df[len(df)-len(val_loader):].index, pred_value, color = 'red', label = 'Predicted Price')
axes.plot(df[len(df)-len(val_loader):].index, true_value, color = 'blue', label = 'Real Price')
#axes.xticks(np.arange(0,394,50))
plt.title('Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
# plt.savefig('ibm_pred.png')
plt.show()